<a href="https://colab.research.google.com/github/Swayam0407/CourseCompass_Web/blob/main/ML_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

........................000000000000000000000000000000000000000000000000000000000000000000000import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving realistic_courses_dataset.csv to realistic_courses_dataset (2).csv


In [ ]:
path='realistic_courses_dataset.csv'
courses_df = pd.read_csv(path)

# Encode categorical variables
le_platform = LabelEncoder()
le_instructor = LabelEncoder()
le_topic = LabelEncoder()
le_certification = LabelEncoder()

courses_df['Platform'] = le_platform.fit_transform(courses_df['Platform'])
courses_df['Instructor Name'] = le_instructor.fit_transform(courses_df['Instructor Name'])
courses_df['Category/Topic'] = le_topic.fit_transform(courses_df['Category/Topic'])
courses_df['Certification (Yes/No)'] = le_certification.fit_transform(courses_df['Certification (Yes/No)'])

# Define features and target
X = courses_df.drop(columns=['Course Title', 'Rating'])
y = courses_df['Rating']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)

# Function to predict the best course for a given topic
def predict_best_course(topic):
    # Check if the topic exists in the training data
    if topic not in le_topic.classes_:
        print(f"Topic '{topic}' not found in the training data.")
        return None

    # Encode the topic
    encoded_topic = le_topic.transform([topic])[0]

    # Filter courses by topic
    topic_courses = courses_df[courses_df['Category/Topic'] == encoded_topic]

    if topic_courses.empty:
        print(f"No courses found for the topic: {topic}")
        return None

    # Prepare the features for prediction
    X_topic = topic_courses.drop(columns=['Course Title', 'Rating'])

    # Predict ratings
    topic_courses['Predicted Rating'] = model.predict(X_topic)

    # Find the course with the highest predicted rating
    best_course = topic_courses.loc[topic_courses['Predicted Rating'].idxmax()]
    return best_course

# Example usage
topic = 'Excel'
best_course = predict_best_course(topic)
if best_course is not None:
    print("Best course for the topic:", topic)
    print(best_course[['Course Title', 'Predicted Rating']])

Best course for the topic: Excel
Course Title        Excel Course 29
Predicted Rating           4.796823
Name: 28, dtype: object


<ipython-input-33-b71d528b6898>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topic_courses['Predicted Rating'] = model.predict(X_topic)
